# Nutrition Calculator  - Australian Food Composition Database

mission statement

To Do:
- Make units kiss
    - Check certain foods
- Add omega 6s properly
- Deal with units and percentage with regex extract to get rid of duplicates and solve above
- Check for duplicates
- Clean df names
- Use classifier trained on recipe file to make appropraite food portions
- Do dot product of foods with DV
    - Adjust weights on nutrients

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set()

In [31]:
# Regex debugger #
pattern = r'\/gN'
df = pd.DataFrame({ 'Lysine \n(mg/gN)': [420, 69], 'Lysine \n(mg)':[1,2]})
new_cols = df.columns.str.contains(pattern)
print(new_cols)

[ True False]


In [35]:
# Import food and nutrition data #

food = pd.read_excel('Australian_Food_Composition_Database/Food Details.xlsx') 
nutrient = pd.read_excel('Australian_Food_Composition_Database/Nutrient file.xlsx',
                         sheet_name='All solids & liquids per 100g')

## Need to drop percent fatty acid content, not necessary for calculation and data cleaning causes repition ##
nutrient = nutrient.drop(columns = ['Total polyunsaturated fatty acids, equated (%T)', 
                          'Total long chain omega 3 fatty acids, equated \n(%T)'])
## Import Daily Value (DV) ##

dv = pd.read_csv('daily value.csv', index_col=0)
dv.head()

,RDI,DV,UL
Nutrient,,,
Calories,2000,2000,NaN
Fat,NaN,78g,NaN
Saturated Fat,NaN,20g,NaN
Cholesterol,NaN,300mg,NaN
Carbs,130g,275g,NaN


In [36]:
## Need to drop percent content columns. Not relevant to goal and causing duplicate columns later on ##
print("Number of columns in df before drop:", len(nutrient.columns))

percent_pattern = r'\(%\w+\)'
percent_cols = nutrient.columns.str.contains(percent_pattern)
print("Number of columns with % in header:", percent_cols.sum())
nutrient = nutrient.drop(columns = nutrient.columns[percent_cols])
print("Number of columns after dropping %:", len(nutrient.columns))

gN_pattern = r'\/gN'
gN_cols = nutrient.columns.str.contains(gN_pattern)
print("Number of columns with /gN in header:", gN_cols.sum())
nutrient = nutrient.drop(columns = nutrient.columns[gN_cols])
print("Number of columns after dropping /gN:", len(nutrient.columns))

Number of columns in df before drop: 291
Number of columns with % in header: 69
Number of columns after dropping %: 222
Number of columns with /gN in header: 18
Number of columns after dropping gN: 204


In [37]:
nutrient.head()

,Public Food Key,Classification,Food Name,"Energy with dietary fibre, equated \n(kJ)","Energy, without dietary fibre, equated \n(kJ)",Moisture (water) \n(g),Protein \n(g),Nitrogen \n(g),"Fat, total \n(g)",Ash \n(g),...,Leucine \n(mg),Lysine \n(mg),Methionine \n(mg),Phenylalanine \n(mg),Proline \n(mg),Serine \n(mg),Threonine \n(mg),Tyrosine \n(mg),Tryptophan \n(mg),Valine \n(mg)
0,F002258,31302.0,"Cardamom seed, dried, ground",1236,1012,8.3,10.8,1.72,6.7,5.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155,NaN
1,F002893,31302.0,"Chilli (chili), dried, ground",1280,1002,10.8,13.4,2.14,14.3,11.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,NaN
2,F002963,31302.0,"Cinnamon, dried, ground",1004,579,10.6,4.0,0.64,1.2,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,NaN
3,F002970,31302.0,"Cloves, dried, ground",1389,1118,9.9,6.0,0.96,13.0,5.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,NaN
4,F003190,31302.0,"Coriander seed, dried, ground",1344,1009,8.9,12.4,1.98,17.8,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,NaN


## Data Cleaning

In [38]:
# Get rid of \n's from headers #
col_pattern = r'\s?\n(.+)'
nutr_cols = nutrient.columns.str.replace(col_pattern, '')
new_cols = dict(zip(nutrient.columns, nutr_cols))
nutrient_clean = nutrient.rename(columns=new_cols)

# Change energy to calories #

fiber=True #toggle this to change how calories from fiber are calculated

if fiber==True:
    energy_cols = {'Energy with dietary fibre, equated': 'Calories'}
    nutrient_clean = nutrient_clean.rename(columns=energy_cols)
    nutrient_clean = nutrient_clean.drop(columns='Energy, without dietary fibre, equated')

else:
    energy_cols = {'Energy, without dietary fibre, equated': 'Calories'}
    nutrient_clean = nutrient_clean.rename(columns=energy_cols)
    nutrient_clean = nutrient_clean.drop(columns='Energy with dietary fibre, equated')

## Convert kJ to kCal ## 
nutrient_clean['Calories'] = nutrient_clean['Calories'].apply(lambda x: x/4.2)  #multiply kJ by this to get kcal

In [39]:
# Extract Units From Cols#

### Attempt 1 - Deleting###
"""The main plan is to delete units in table header. Because every unit in both datasets are metric, an inconsistency 
in units would manifest as a 3 orders of magnitude off which will be investigated in further analysis."""

# Get rid of (abbrev.)#
col_pattern = r' \(.+\)'
nutr_cols = nutrient_clean.columns.str.replace(col_pattern, '')
new_cols = dict(zip(nutrient_clean.columns, nutr_cols))
nutrient_clean = nutrient_clean.rename(columns=new_cols)


In [40]:
# Need to change column names to be equivalent between food and DV #

## Vitamins ##
vitamins = {'Thiamin': 'Vitamin B1',
'Riboflavin': 'Vitamin B2',
'Niacin derived equivalents': 'Vitamin B3', 
'Pantothenic acid':'Vitamin B5',
'Pyridoxine':'Vitamin B6',
'Biotin': 'Vitamin B7',
'Total folates': 'Vitamin B9',
'Cobalamin': 'Vitamin B12',
'Vitamin A retinol equivalents':'Vitamin A',
'Vitamin D3 equivalents': 'Vitamin D'
           }
nutrient_clean = nutrient_clean.rename(columns=vitamins)

## Fat ##
nutrient_clean = nutrient_clean.rename(columns = {'Fat, total': 'Fat', 
                                                    'Total saturated fatty acids, equated': 'Saturated Fat',
                                                   'Total long chain omega 3 fatty acids, equated': 'Omega 3s'
                                                   })

### Adding omega 6s from manual Calc ###
### Assuming any polyunsaturated fat that isn't omega-3 is an omega-6
nutrient_clean['Omega 6s'] = round(nutrient_clean['Total polyunsaturated fatty acids, equated'] - nutrient_clean['Omega 3s']*0.001,2)

## Carbohydrates ##
carbs = {'Total sugars':'Sugar',
          'Total dietary fibre':'Fiber',
        'Available carbohydrate, with sugar alcohols':'Carbs'}
nutrient_clean = nutrient_clean.rename(columns=carbs)

## Misc Nutrients ##

misc_nutr = {'Cystine plus cysteine':'Cystine',
            }
nutrient_clean = nutrient_clean.rename(columns=misc_nutr)


In [41]:
def shared_columns(df1, df2):
    
    """Finds which columns in df1 are and are not shared with df2"""
    
    df1_cols = df1.columns
    df2_cols = df2.columns
    
    shared_cols = []
    diff_cols = []
    
    for cols in df1_cols:
        if cols in df2_cols:
            shared_cols.append(cols)
        else:
            diff_cols.append(cols)
    
    return shared_cols, diff_cols

In [42]:
## Find DV columns that aren't equal to nutrient cols ##

dv_clean = dv.rename(index={'Lutein+zeazanthin': 'Lutein'})
nutr_cols = nutrient_clean.columns
dv_shared = shared_columns(dv_clean.transpose(), nutrient_clean)


"""This data does not exist in the food list df, therefore we will ignore these nutrients. Water is a 
nutrient that should be satisfied outside of food consumption. Therefore, it should be excluded
from the analysis. Furthermore, the remaining incongruent sets are non-essential amino acids which can
be produced in the human body, making it less important for consumption. Vitamin K and Choline
are essential in the human diet. However, the food data set does not have this included. This data could 
be imputed in the event of expanding the model for a more robust study."""

dv_clean = dv_clean.drop(index=dv_shared[1])
print("Dropping columns:", dv_shared[1])
dv_shared_dropped = shared_columns(dv_clean.transpose(), nutrient_clean)

print("Shared columns:", len(dv_shared_dropped[0]),"\nDifferent columns:", len(dv_shared_dropped[1]))



Dropping columns: ['Vitamin K', 'Choline', 'Water', 'Aspartic Acid', 'Glutamic Acid', 'Hydroxyproline']
Shared columns: 58 
Different columns: 0


In [43]:
# Extract Units From DV #

unit_pattern = r'\d+(\w+)'
number_pattern = r'(\d+(.?\d+))'
dv_clean['RDI units'] = dv_clean['RDI'].str.extract(unit_pattern)
dv_clean['RDI'] = dv_clean['RDI'].str.extract(number_pattern)
dv_clean['DV units'] = dv_clean['DV'].str.extract(unit_pattern)
dv_clean['DV'] = dv_clean['DV'].str.extract(number_pattern)

In [44]:
dv_clean.head()

,RDI,DV,UL,RDI units,DV units
Nutrient,,,,,
Calories,2000,2000,NaN,0,0
Fat,NaN,78,NaN,NaN,g
Saturated Fat,NaN,20,NaN,NaN,g
Cholesterol,NaN,300,NaN,NaN,mg
Carbs,130,275,NaN,g,g


In [45]:
nutrient_clean.head()

,Public Food Key,Classification,Food Name,Calories,Moisture,Protein,Nitrogen,Fat,Ash,Fiber,...,Lysine,Methionine,Phenylalanine,Proline,Serine,Threonine,Tyrosine,Tryptophan,Valine,Omega 6s
0,F002258,31302.0,"Cardamom seed, dried, ground",294.285714,8.3,10.8,1.72,6.7,5.8,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155,NaN,1.39
1,F002893,31302.0,"Chilli (chili), dried, ground",304.761905,10.8,13.4,2.14,14.3,11.8,34.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,NaN,6.69
2,F002963,31302.0,"Cinnamon, dried, ground",239.047619,10.6,4.0,0.64,1.2,3.6,53.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,NaN,0.12
3,F002970,31302.0,"Cloves, dried, ground",330.714286,9.9,6.0,0.96,13.0,5.6,33.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,NaN,4.76
4,F003190,31302.0,"Coriander seed, dried, ground",320.000000,8.9,12.4,1.98,17.8,6.0,41.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,NaN,1.82


In [46]:
a = {}
for nut in nutrient_clean.columns:
    if nut in a:
        a[nut] +=1
    else:
        a[nut] = 1

In [50]:
b = 0
for keys, vals in a.items():
    if vals > 1:
        print('Repeat column')
        b+=1
if b == 0:
    print('No repeat columns')

No repeat columns
